In [22]:
from queue import Queue
import time

In [23]:
def read_input(filename):
    with open(filename, 'r') as f:
        d = {}
        for (i, l) in enumerate(f.readlines()):
            for (j, ll) in enumerate(l[:-1]):
                d[(i, j)] = ll
    return d

def get_key(d, val):
    for (key, value) in d.items():
        if value == val:
            return key
    return 'Start not found'

In [38]:
def get_adjacent(p, tunnel_map, all_keys):
    x, y = p[0]
    doors = [k.upper() for k in p[1]]
    adj = []
    for p_a in [(x - 1, y), (x + 1, y), (x, y - 1), (x, y + 1)]:
        if tunnel_map[p_a] in ['.', '@'] + doors + all_keys:
            adj.append((p_a, p[1]))
    return adj

def is_explored(explored, w):
    return any([w[0] == e[0] and set(w[1]) == set(e[1]) for e in explored])

def bfs(tunnel_map, p_start, all_keys):
    q = Queue()
    q.put(p_start)
    explored = []
    dist = {p_start: 0}
    while not q.empty():
        v = q.get()
        #if v in dist and v != p_start:
        #    continue
        xy = v[0]
        if tunnel_map[xy] in all_keys:
            if not tunnel_map[xy] in v[1]:
                v_old = v
                v = (v[0], v[1] + tunnel_map[xy])
                dist[v] = dist[v_old]
        if set(v[1]) == set(all_keys):
            return dist[v]
        for w in get_adjacent(v, tunnel_map, all_keys):
            if not is_explored(explored, w):
                explored.append(w)
                dist[w] = dist[v] + 1
                q.put(w)
    return -1


In [37]:
explored = [((2, 3), 'abcde'), ((3, 3), 'fbcde')]
is_explored(explored, ((2, 3), 'bdeca'))

True

In [39]:
def runit(filename):
    tunnel_map = read_input(filename)
    all_keys = [c for c in list(set(tunnel_map.values()) - set(['#', '.', '@'])) if c.islower()]
    p_start = get_key(tunnel_map, '@')
    t0 = time.time()
    dist = bfs(tunnel_map, (p_start, ''), all_keys)
    dt = time.time() - t0
    res = {'time': dt, 'distance': dist}
    return res

In [45]:
res = runit('18_input.txt')
res

KeyboardInterrupt: 